## PFG Bank: Credit Card Design

* Team-lead GitLab id: 243
* Group name: Korinna
* Team member names: Siqi Chen, Wen-Hsuan Hung, Xinyu Lou, Yuefeng Mao

## Setup

Please complete this Jupyter notebook by answering the questions in `pfg-bank.pdf` on Canvas (week10/). Create a Notebook and HTML file with all your results and comments and push both the Notebook and HTML file to GitLab when your team is done. All results MUST be reproducible (i.e., the TA and I must be able to recreate the HTML from the Jupyter Notebook file without changes or errors). This means that you should NOT use any python-packages that are not part of the rsm-msba-spark docker container.

This is the final group assignment for MGTA 455 and you will be using git and GitLab. If two people edit the same file at the same time you could get what is called a "merge conflict". git will not decide for you who's change to accept so the team-lead will have to determine which edits to use. To avoid merge conflicts, **always** "pull" changes to the repo before you start working on any files. Then, when you are done, save and commit your changes, and push them to GitLab. Make "pull first" a habit!

In [1]:
import os

import numpy as np
import pandas as pd
import pyrsm as rsm
import statsmodels.formula.api as smf
from statsmodels.genmod.families import Binomial
from statsmodels.genmod.families.links import logit

### 1. Historical Data Analysis

In [2]:
data = pd.read_excel("data/exhibits.xls")

In [3]:
data_combine = pd.melt(
    data,
    id_vars=["date", "apr","fixed_var","annual_fee","visamc","bk_score","average_bk"],
    value_vars=["resp", "non_resp"],
    var_name="response",
    value_name="freq",
)

In [4]:
data_combine["resp_sale"] = rsm.ifelse(data_combine.response == "resp", 1, 0)
data_combine = data_combine.drop(["visamc","date","average_bk"],axis=1)

In [5]:
## try logistic regression without interactions
lr = smf.glm(
    formula="resp_sale ~ apr + fixed_var + annual_fee + bk_score",
    family=Binomial(link=logit()),
    data=data_combine,
    freq_weights=data_combine.freq,
).fit()
rsm.or_ci(lr)

,index,OR,OR%,2.5%,97.5%,p.values,
1,fixed_var[T.Variable],0.827,-17.3%,0.796,0.860,< .001,***
2,apr,0.763,-23.7%,0.756,0.769,< .001,***
3,annual_fee,0.941,-5.9%,0.939,0.942,< .001,***
4,bk_score,1.003,0.3%,1.003,1.003,< .001,***


In [6]:
rsm.model_fit(lr)


Pseudo R-squared (McFadden): 0.032
Pseudo R-squared (McFadden adjusted): 0.032
Area under the RO Curve (AUC): 0.5
Log-likelihood: -133068.981, AIC: 266147.961, BIC: 266209.132
Chi-squared: 1493244.41 df(4), p.value < 0.001 
Nr obs: 28



In [7]:
### try add interactions 
lr_inter = smf.glm(
    formula="resp_sale ~ apr + fixed_var + annual_fee + bk_score + apr:fixed_var + \
                         apr:annual_fee + fixed_var:annual_fee",
    family=Binomial(link=logit()),
    data=data_combine,
    freq_weights=data_combine.freq,
).fit()
rsm.or_ci(lr_inter)

,index,OR,OR%,2.5%,97.5%,p.values,
1,fixed_var[T.Variable],0.999,-0.1%,0.999,1.000,< .001,***
2,apr,0.818,-18.2%,0.809,0.827,< .001,***
3,apr:fixed_var[T.Variable],0.992,-0.8%,0.992,0.993,< .001,***
4,annual_fee,1.093,9.3%,1.077,1.110,< .001,***
5,fixed_var[T.Variable]:annual_fee,0.990,-1.0%,0.989,0.991,< .001,***
6,bk_score,1.002,0.2%,1.002,1.002,< .001,***
7,apr:annual_fee,0.991,-0.9%,0.990,0.992,< .001,***


In [8]:
rsm.model_fit(lr_inter)


Pseudo R-squared (McFadden): 0.034
Pseudo R-squared (McFadden adjusted): 0.034
Area under the RO Curve (AUC): 0.5
Log-likelihood: -132865.835, AIC: 265743.671, BIC: 265817.076
Chi-squared: 1536295.008 df(5), p.value < 0.001 
Nr obs: 28



All of interactions are significant and the lr_inter's Pseudo R-squared was improved. Therefore, we decided to use the model with interactions.

In [9]:
dct = rsm.levels_list(data_combine[["apr", "fixed_var", "annual_fee","bk_score"]])
data_combine_expand = rsm.expand_grid(dct)

In [10]:
data_combine_expand["pred_all"] = lr_inter.predict(data_combine_expand)

In [11]:
clv = pd.read_excel("data/exhibits.xls",sheet_name="exhibit2")

In [12]:
data_combine_expand_clv = data_combine_expand.merge(clv,how="left",on=['apr','fixed_var','annual_fee'])

In [13]:
data_combine_expand_clv['clv'] = rsm.ifelse(data_combine_expand_clv["bk_score"]==150, data_combine_expand_clv["clv150"],\
                                          rsm.ifelse(data_combine_expand_clv["bk_score"]==200, data_combine_expand_clv["clv200"],\
                                                     data_combine_expand_clv["clv250"]))

In [14]:
data_combine_expand_clv['profit'] = data_combine_expand_clv['clv']*data_combine_expand_clv['pred_all']
idx = data_combine_expand_clv.groupby(['bk_score'])['profit'].transform(max)==data_combine_expand_clv['profit']
data_combine_expand_clv[idx]

,apr,fixed_var,annual_fee,bk_score,pred_all,offer,clv150,clv200,clv250,clv,profit
9,16.8,Variable,0,200,0.032054,8,82,62,32,62,1.987341
22,19.8,Variable,0,250,0.019141,12,110,90,60,60,1.148458
35,14.9,Variable,0,150,0.042761,4,62,42,12,62,2.651211


The dataframe above shows the one best offer for each type of bk_score based on historical data.

### 2. Decide round 1 strategy
Next, we have to decide whether we should send out emails to all 36 groups of customers, or only test part of the customers in round 1. The advantage of factorial design is that we can predict some groups' response rate using the result of round 1, so that we are able to generate a higher profit from round 2 targeting. However, we might not be able to understand the real response for each group of customers. Thus, we conduct DOE to see how many trials we need so that the partial data can represent the full size data set.

#### 1) Decide number of trials in partial factorial design
The result shows that when trial number is 18, d-efficiency is 0.965, which is higher than the 0.9 threshold we set. Therefore we need 18 trials to represent the total 36 combinations of product and customer.


In [15]:
%reload_ext rpy2.ipython

In [16]:
%%R 
result <- radiant.design::doe(
  factors = c(
    "annual_fee; 0; 20 ", 
    "fixed_var; fixed; variable ", 
    "apr; 14.9; 16.8; 19.8 ", 
    "bk_score; 150; 200; 250 "
  ), 
  seed = 1237
)
summary(result, eff = TRUE, part = TRUE, full = TRUE)
# write.csv(result$part, file = "part_factorial.csv")

Experimental design
# trials for partial factorial: 36 
# trials for full factorial   : 36 
Random seed                   : 1237 

Attributes and levels:
annual_fee: 0, 20 
fixed_var: fixed, variable 
apr: 14.9, 16.8, 19.8 
bk_score: 150, 200, 250_ 

Design efficiency:
 Trials D-efficiency Balanced
      7        0.368    FALSE
      8        0.276    FALSE
      9        0.751    FALSE
     10        0.705    FALSE
     11        0.507    FALSE
     12        0.892     TRUE
     13        0.813    FALSE
     14        0.792    FALSE
     15        0.726    FALSE
     16        0.678    FALSE
     17        0.622    FALSE
     18        0.965     TRUE
     19        0.911    FALSE
     20        0.869    FALSE
     21        0.821    FALSE
     22        0.852    FALSE
     23        0.848    FALSE
     24        0.922     TRUE
     25        0.883    FALSE
     26        0.846    FALSE
     27        0.914    FALSE
     28        0.888    FALSE
     29        0.862    FALSE
     30   

#### 2) Decide which groups to test in the experiment
In this step, we need to check which groups to send out round 1 mailing in the experiment. We set trials to 18, random seed to 5, and get the result as below. These are the groups of people that we are going to target in round 1. 

We also tried setting different seeds, like seed 1234 and 172110, and found that it will recommend us to experiment on different groups. To choose which seed we should set to select the trials, we based our results on the model we trained on historical data. We got different combinations of expected profit, and we found that the trials seed 5 covers all of the product/customer combinations with the highest profits calculated through historical data. Thus, we decide to select the groups from the result of seed 5.

In [17]:
%%R 
result <- radiant.design::doe(
  factors = c(
    "annual_fee; 0; 20 ", 
    "fixed_var; fixed; variable ", 
    "apr; 14.9; 16.8; 19.8 ", 
    "bk_score; 150; 200; 250"
  ), 
  trials = 18, 
  seed = 5
)
summary(result, eff = TRUE, part = TRUE, full = TRUE)
# write.csv(result$part, file = "part_factorial.csv")

Experimental design
# trials for partial factorial: 18 
# trials for full factorial   : 36 
Random seed                   : 5 

Attributes and levels:
annual_fee: 0, 20 
fixed_var: fixed, variable 
apr: 14.9, 16.8, 19.8 
bk_score: 150, 200, 250 

Design efficiency:
 Trials D-efficiency Balanced
     18        0.965     TRUE

Partial factorial design correlations:
** Note: Variables are assumed to be ordinal **
           annual_fee fixed_var apr bk_score
annual_fee      1.000     0.174   0        0
fixed_var       0.174     1.000   0        0
apr             0.000     0.000   1        0
bk_score        0.000     0.000   0        1

Partial factorial design:
 trial annual_fee fixed_var  apr bk_score
     3          0     fixed 14.9      250
     4          0     fixed 16.8      150
     6          0     fixed 16.8      250
     7          0     fixed 19.8      150
     8          0     fixed 19.8      200
    10          0  variable 14.9      150
    11          0  variable 14.9      20

In [18]:
%%R 
result <- radiant.design::doe(
  factors = c(
    "annual_fee; 0; 20 ", 
    "fixed_var; fixed; variable ", 
    "apr; 14.9; 16.8; 19.8 ", 
    "bk_score; 150; 200; 250"
  ), 
  seed = 1237
)
summary(result, eff = TRUE, part = TRUE, full = TRUE)
write.csv(result$part, file = "part_factorial.csv")

Experimental design
# trials for partial factorial: 36 
# trials for full factorial   : 36 
Random seed                   : 1237 

Attributes and levels:
annual_fee: 0, 20 
fixed_var: fixed, variable 
apr: 14.9, 16.8, 19.8 
bk_score: 150, 200, 250 

Design efficiency:
 Trials D-efficiency Balanced
      7        0.368    FALSE
      8        0.276    FALSE
      9        0.751    FALSE
     10        0.705    FALSE
     11        0.507    FALSE
     12        0.892     TRUE
     13        0.813    FALSE
     14        0.792    FALSE
     15        0.726    FALSE
     16        0.678    FALSE
     17        0.622    FALSE
     18        0.965     TRUE
     19        0.911    FALSE
     20        0.869    FALSE
     21        0.821    FALSE
     22        0.852    FALSE
     23        0.848    FALSE
     24        0.922     TRUE
     25        0.883    FALSE
     26        0.846    FALSE
     27        0.914    FALSE
     28        0.888    FALSE
     29        0.862    FALSE
     30    

In [19]:
%%R 
result <- radiant.design::doe(
  factors = c(
    "annual_fee; 0; 20 ", 
    "fixed_var; fixed; variable ", 
    "apr; 14.9; 16.8; 19.8 ", 
    "bk_score; 150; 200; 250"
  ), 
  trials = 18, 
  seed = 172110
)
summary(result, eff = TRUE, part = TRUE, full = TRUE)
# write.csv(result$part, file = "part_factorial.csv")

Experimental design
# trials for partial factorial: 18 
# trials for full factorial   : 36 
Random seed                   : 172110 

Attributes and levels:
annual_fee: 0, 20 
fixed_var: fixed, variable 
apr: 14.9, 16.8, 19.8 
bk_score: 150, 200, 250 

Design efficiency:
 Trials D-efficiency Balanced
     18        0.965     TRUE

Partial factorial design correlations:
** Note: Variables are assumed to be ordinal **
           annual_fee fixed_var apr bk_score
annual_fee      1.000    -0.174   0        0
fixed_var      -0.174     1.000   0        0
apr             0.000     0.000   1        0
bk_score        0.000     0.000   0        1

Partial factorial design:
 trial annual_fee fixed_var  apr bk_score
     2          0     fixed 14.9      200
     4          0     fixed 16.8      150
     5          0     fixed 16.8      200
     9          0     fixed 19.8      250
    10          0  variable 14.9      150
    12          0  variable 14.9      250
    14          0  variable 16.8   

#### 3) Define Sample Size
To decide how big the sample size should be, we conduct power analysis. In the analysis, we assign p1 = 2%, p2 = 3%, the result shows that we need at least 3,789 samples to see whether we can accept or decline the null nypothesis of p1 = p2. Since we want to go for a more conservative strategy, and ensure that we have enough data for the experiment, we decide to increase the sample size to 4000 for the experiment.

In [29]:
%%R 
result <- radiant.design::sample_size_comp(
  type = "proportion", 
  p1 = 0.02, 
  p2 = 0.03, 
  conf_lev = 0.95, 
  power = 0.8
)
summary(result)

Sample size calculation for comparison of proportions 
Sample size 1    : 3,789
Sample size 2    : 3,789
Total sample size: 7,578
Proportion 1     : 0.02 
Proportion 2     : 0.03 
Effect size      : 0.06437191 
Confidence level : 0.95 
Power            : 0.8 
Alternative      : two.sided 



### 3. Round1 Data Anylysis

In [21]:
round1 =  pd.read_excel("round1result_pre.xls")

In [22]:
round1 = round1.drop(round1[round1['Sent']==0].index)
round1['non_res']=round1['Sent']-round1['Responses']

In [23]:
round1_combine = pd.melt(
    round1,
    id_vars=[ "APR","Fixed/Var.","Annual Fee","bk_score"],
    value_vars=["Responses", "non_res"],
    var_name="response",
    value_name="freq",
)
round1_combine["resp_sale"] = rsm.ifelse(round1_combine.response == "Responses", 1, 0)
round1_combine = round1_combine.rename({'APR':'apr','Fixed/Var.':'fixed_var','Annual Fee':'annual_fee'},axis=1)
round1_combine['apr'] = round1_combine['apr'].astype('category')
round1_combine['annual_fee'] = round1_combine['annual_fee'].astype('category')
round1_combine['fixed_var'] = round1_combine['fixed_var'].astype('category')

In [24]:
### try fit model with round1 data
lr_inter = smf.glm(
    formula="resp_sale ~ apr + fixed_var + annual_fee + bk_score + apr:fixed_var + \
                         apr:annual_fee + fixed_var:annual_fee",
    family=Binomial(link=logit()),
    data=round1_combine,
    freq_weights=round1_combine.freq,
).fit()
rsm.or_ci(lr_inter)

,index,OR,OR%,2.5%,97.5%,p.values,
1,apr[T.16.8],0.430,-57.0%,0.350,0.528,< .001,***
2,apr[T.19.8],0.271,-72.9%,0.207,0.355,< .001,***
3,fixed_var[T.Var.],0.820,-18.0%,0.655,1.027,0.084,.
4,annual_fee[T.20],0.400,-60.0%,0.331,0.484,< .001,***
5,apr[T.16.8]:fixed_var[T.Var.],1.017,1.7%,0.742,1.394,0.918,
6,apr[T.19.8]:fixed_var[T.Var.],1.148,14.8%,0.737,1.789,0.542,
7,apr[T.16.8]:annual_fee[T.20],0.799,-20.1%,0.571,1.117,0.189,
8,apr[T.19.8]:annual_fee[T.20],0.641,-35.9%,0.420,0.976,0.038,*
9,fixed_var[T.Var.]:annual_fee[T.20],0.737,-26.3%,0.551,0.986,0.04,*
10,bk_score,1.004,0.4%,1.002,1.007,< .001,***


In [25]:
### delete all insignificant items
lr = smf.glm(
    formula="resp_sale ~ apr + annual_fee + bk_score",
    family=Binomial(link=logit()),
    data=round1_combine,
    freq_weights=round1_combine.freq,
).fit()
rsm.or_ci(lr)

,index,OR,OR%,2.5%,97.5%,p.values,
1,apr[T.16.8],0.404,-59.6%,0.355,0.461,< .001,***
2,apr[T.19.8],0.253,-74.7%,0.217,0.296,< .001,***
3,annual_fee[T.20],0.327,-67.3%,0.288,0.371,< .001,***
4,bk_score,1.006,0.6%,1.005,1.007,< .001,***


Based on the significance of each item, we decided to use the model without interactions.

In [27]:
dct = rsm.levels_list(round1_combine[["apr", "fixed_var", "annual_fee","bk_score"]])
round1_combine_expand = rsm.expand_grid(dct)
round1_combine_expand["pred_all"] = lr.predict(round1_combine_expand)
clv = clv.replace({'Variable':'Var.'})
round1_combine_expand_clv = round1_combine_expand.merge(clv,how='left',on=['apr','fixed_var','annual_fee'])
round1_combine_expand_clv['clv'] = rsm.ifelse(round1_combine_expand_clv["bk_score"]==150, round1_combine_expand_clv["clv150"],\
                                          rsm.ifelse(round1_combine_expand_clv["bk_score"]==200, round1_combine_expand_clv["clv200"],\
                                                     round1_combine_expand_clv["clv250"]))
round1_combine_expand_clv['profit'] = round1_combine_expand_clv['clv']*round1_combine_expand_clv['pred_all']
idx = round1_combine_expand_clv.groupby(['bk_score'])['profit'].transform(max)==round1_combine_expand_clv['profit']
round1_combine_expand_clv[idx]

,apr,fixed_var,annual_fee,bk_score,pred_all,offer,clv150,clv200,clv250,clv,profit
3,14.9,Var.,0,150,0.035889,4,62,42,12,62,2.225118
4,14.9,Var.,0,200,0.047556,4,62,42,12,42,1.997364
29,19.8,Var.,0,250,0.016690,12,110,90,60,60,1.001378


Based on our model's predication, we decided to offer as follow in round2

`BK_score = 150` : <14.9 Var. 0>; 

`BK_score = 200` : <14.9 Var. 0>;

`BK_score = 250` : <19.8 Var. 0>.

### 4. Further Analysis 

#### 1) Analysis of ClV variation

According to exhibit 2, we see that for people with higher BK score generally have lower CLV. This is because when people go bankrupt, they are not able to keep their credit card. Thus as the BK score increases, the risk of going bankrupt increases, and the CLV decreases. 

The reason that CLV varies by product is because each product has different attributes. From exhibit 2, we can see that PFG generates higher CLV for higher APR. This is because higher APR refers to higher finance charges that a customer accrued on outstanding balances. On one hand, higher APR directly brings more profits to PFG. On the other hand, higher APR indicates that customers are insensitive to charges, or they depend much more on credit cards than other groups, so they are less likely to churn in the future. 

Other than that, customers with variable-rate cards generate higher CLV than those who have fixed-rate cards. With variable-rate cards, PFG could pass along most of the interest-rate risk to their customers since a rise in a bank’s borrowing costs lead to a rise in the price to the customer, which protects the margin and hence improves the CLV. 

As for the annual fee, customers who pay 20 each year bring higher CLV than those who pay 0. Just like the principle behind APR, a higher annual fee not only increases the revenue, but also indicates that customers have higher fidelity and dependence on credit cards and are less likely to churn.



#### 2) Analysis of historical data

Historic data indeed provides us with some insights regarding past customer responses for different product combinations, which can be helpful when deciding which solicitations to target in our trial run. By building a logistic regression model based on available historic data, we can see that for BK Score 150, the profit-maximizing solicitation is 14.9 (Apr), Variable (fixed_var) and 0 (annual fee); 16.8, Variable, 0 for BK Score 200; and 19.8, Variable, 0 for BK Score 250. With this information, we can increase the profit of our trial run by finding out a random seed that can generate partial factorial design that contains these three different solicitations. After some testings, we found when seed = 5, the 18 partial factorial designs generated contain the previously mentioned three products. Therefore, historic data can provide us some insights when determining trial round targets. 

However, historical data might not be completely reliable when building a predictive logistic regression model. The reason that predictive model based on historical data is not too useful in this case is:

1). The market changes as the time changes, thus we are not able to accurately predict the future based on past data. 

2). Second, the historical data is collected from different months of a year, and each month has data for different product lines. Therefore, the impact from products on customers may be mixed with the impact from months. Since we cannot accurately determine the effect that time of year has on customer responses, it is hard to measure how different products influence customer responses.

3). After the last mailing campaign, many factors beyond PFG’s control had changed and could affect customers’ responses. Several competitors of PFG had launched major marketing campaigns during the holiday season. One competitor was aggressively marketing no-fee cards and a second was offering a substantial rebate program. In addition, interest rates had dropped: in the past six months, PFG’s cost of borrowing had fallen almost a full percentage point. All these factors may exert influence on the behaviors and response willingness of PFG’s customers, so we need to conduct a new mailing campaign to better analyze their customers.


#### 3) Analysis of "best product"

In [30]:
oneoffer_profit = pd.DataFrame(data_combine_expand_clv.groupby(['apr','fixed_var','annual_fee'])['profit'].sum())
oneoffer_profit

profit
apr  fixed_var annual_fee          
14.9 Fixed     0           4.263901
               20          3.806209
     Variable  0           5.239079
               20          3.270693
16.8 Fixed     0           5.138287
               20          2.505939
     Variable  0           5.506319
               20          2.037000
19.8 Fixed     0           4.555690
               20          1.091587
     Variable  0           4.454458
               20          0.839758

In [31]:
best = oneoffer_profit[oneoffer_profit.profit == max(oneoffer_profit.profit)]
best

,,,profit
apr,fixed_var,annual_fee,
16.8,Variable,0,5.506319


If we send the same product promotion mails to all customers, based on Historical Data, <16.8 Var 0> may be the 'best' offer, then let's calculate the profit if we send it to all customers.

In [32]:
fix_cost = 10000
production_cost = 1000
var_cost = 0.5
round_cost = 800
profit_one = best.reset_index()['profit'][0]*250000 - fix_cost - 1* production_cost-750000*var_cost-round_cost
profit_one

989779.8281505387

Based on Historical Data,if we customize for different type of bk_score

In [33]:
customize = data_combine_expand_clv[idx]
customize 

,apr,fixed_var,annual_fee,bk_score,pred_all,offer,clv150,clv200,clv250,clv,profit
3,16.8,Fixed,0,200,0.036258,7,72,52,22,52,1.885435
4,16.8,Fixed,0,250,0.039807,7,72,52,22,22,0.875753
29,14.9,Fixed,0,150,0.047644,3,52,32,2,52,2.477484


In [34]:
profit_customize= customize['profit'].sum()*250000 - fix_cost - 3* production_cost-750000*var_cost-round_cost
profit_customize

920867.9977529331

According to calculation, the customize way is better than one-offer way, which means there is no 'best' offer for all of customers.

#### 4) Describe and justify the testing strategy
We divide our testing process into 2 rounds.

In round 1, our objective is to gain enough data to 1). indicate the actual response rate of each product and each group of customers, 2). represent the full factorial design using only partial trials, 3). Cover as many profitable trials as possible when the first 2 conditions are met. 

To meet the first condition, we conduct power analysis in Radiant. In this case, we assume that a difference level of 1% in response rate makes sense in our decision making. With the sample size calculation tool in Radiant, we assign p1 = 2%, p2 = 3%, the result shows that we need at least 3,789 samples to see whether we can accept or decline the null hypothesis of p1 = p2. Since we want to go for a more conservative strategy, and ensure that we have enough data for the experiment, we decide to increase the sample size to 4000 for the experiment.

And to meet the second and third conditions, we first train a logistic regression model based on historical data and calculate the probability of receiving offer for each product and each bk group of customers. Given their CLV, we then calculate the expected profit of sending different offers for each bk group. In the next step, we try different seeds in the DOE tool. With different seeds, the partial factorial design with 18 trials always has the highest D-efficiency of 0.965, and the 18 trials in seed 5 results cover the most number of profitable products. Hence we decide to send 4000 mails for each of these 18 combinations.

And then, with results from round 1, we re-estimate our logistic regression model and calculate the predicted probabilities and expected profit again. For round 2, since we already gained enough data from round1, for each bk group, we only send the offer which generates the highest expected profit to them.
